## Tutorial 2: A Complete HoloClean Pipeline

In this tutorial, we will walk step-by-step through the process of repairing a dataset in `HoloClean`. 

The dataset in question contains information about hospitals and is commonly-used for research purposes. Errors are present in ~5% of the cells and there is significant duplicate information - the ideal environment for `HoloClean`.

### Step 1: Data Loading

We begin by instantiating the `HoloClean` and `Session` objects needed to run the repairs. For a more detailed overview of these objects and the rest of our infrastructure, please see our [Data Loading & Denial Constraints Tutorial](Tutorial_1.ipynb).
To altar exposed variables such as pruning threshold. 
Send as a parameter when initializing the HoloClean object.
Examples of exposed variables:
    Learning Rate: learning_rate
    Pruning Threshold: pruning_threshold
    Number of Learning Iterations: learning_iterations
    Batch Size: batch_size
For a list of all possible arguements check the holoclean.py file

In [1]:
from holoclean.holoclean import HoloClean, Session

holo =      HoloClean(
            holoclean_path="..",
            verbose=True,
            timing_file='execution_time.txt',
            learning_iterations=50,
            learning_rate=0.001,
            batch_size=20)
session = Session(holo)

/home/gmichalo/anaconda/envs/python27/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Next, we load in the data and denial constraints needed for this dataset. Both pieces of information are stored in the MySQL database.

In [2]:
data_path = "data/hospital_dataset.csv"

## loads data into our database and returns pyspark dataframe of initial data
data = session.load_data(data_path)

dc_path = "data/hospital_constraints.txt"

# loads denial constraints into our database and returns a simple list of dcs as strings 
dcs = session.load_denial_constraints(dc_path)

Time to Load Data: 6.18669819832



It's easy to see the dataset has a decent amount of errors. Note the random 'x' characters that have been substituted in.

In [3]:
# all pyspark dataframe commands available
data.select('City').show(15)

+----------+
|      City|
+----------+
|BIRMINGHAM|
|BIRMINGHAM|
|BIRMINGHAM|
|BIRMINGHxM|
|BIRMINGHAM|
|BIRMINGHAM|
|BIRMINGHAM|
|BIRMINGxAM|
| SHEFFIELD|
| SHEFFIELD|
| SHEFFxELD|
| SHEFFIELD|
| SHEFFIELD|
| SHEFFIELD|
| SHEFFIELD|
+----------+
only showing top 15 rows



In [4]:
# a simple list of strings
dcs

['t1&t2&EQ(t1.ZipCode,t2.ZipCode)&IQ(t1.City,t2.City)',
 't1&t2&EQ(t1.ZipCode,t2.ZipCode)&IQ(t1.State,t2.State)',
 't1&t2&EQ(t1.PhoneNumber,t2.PhoneNumber)&IQ(t1.ZipCode,t2.ZipCode)',
 't1&t2&EQ(t1.PhoneNumber,t2.PhoneNumber)&IQ(t1.City,t2.City)',
 't1&t2&EQ(t1.PhoneNumber,t2.PhoneNumber)&IQ(t1.State,t2.State)',
 't1&t2&EQ(t1.ProviderNumber,t2.ProviderNumber)&EQ(t1.MeasureCode,t2.MeasureCode)&IQ(t1.Stateavg,t2.Stateavg)',
 't1&t2&EQ(t1.MeasureCode,t2.MeasureCode)&IQ(t1.MeasureName,t2.MeasureName)',
 't1&t2&EQ(t1.MeasureCode,t2.MeasureCode)&IQ(t1.Condition,t2.Condition)',
 't1&t2&EQ(t1.State,t2.State)&EQ(t1.MeasureCode,t2.MeasureCode)&IQ(t1.Stateavg,t2.Stateavg)']

### Step 2: Error Detection

HoloClean is a supervised error repairing system. In contrast to traditional supervision, we do not ask users to label individual data cells but rely on more high-level supervision signals. These signals are denial constraints or other, custom-made error detectors that split the data into two categories, "clean" and "don't-know". Using that split, our later steps of the process will be able to learn the features of a "clean" cell and perform inference on the values of the "don't-know" cells.

Please see our <a href=http://pages.cs.wisc.edu/~thodrek/blog/holoclean.html>blog post</a> for more information

In this tutorial, we will use HoloClean's built in error detector that uses denial constraints to perform this split. Any cell that participates in a violation of a DC is marked "don't-know", the rest are treated as clean. If you wish to develop a custom error detector, please see our [Error Detectors Tutorial](Tutorial_3.ipynb) for a walkthrough.

In [5]:
from holoclean.errordetection.mysql_dcerrordetector import MysqlDCErrorDetection

# instantiate Holoclean's built in error detector
detector = MysqlDCErrorDetection(session)

# both clean and dirty sets are returned as pyspark dataframes
error_detector_list =[]
error_detector_list.append(detector)
clean, dirty = session.detect_errors(error_detector_list)

Time for Error Detection: 14.6848258972



In [6]:
clean.head(5)

[Row(ind=3, attr=u'Score'),
 Row(ind=22, attr=u'Sample'),
 Row(ind=28, attr=u'Address3'),
 Row(ind=30, attr=u'Address2'),
 Row(ind=38, attr=u'Sample')]

In [7]:
dirty.head(5)

[Row(ind=471, attr=u'ZipCode'),
 Row(ind=703, attr=u'ZipCode'),
 Row(ind=314, attr=u'ZipCode'),
 Row(ind=47, attr=u'City'),
 Row(ind=304, attr=u'City')]

### Step 3: Repairing

With the "clean" and "don't-know" split defined, we are ready to perform repairs.

Denial Constraints are the driving force behind this process. Denial constraints are used as features in a softmax regression model. The clean cells are used as training examples to learn the parameters (weights) of this model. Once those weights are defined, we use this model to perform inference on the "don't-know" cells and insert the most likely value for each cell.

This tutorial will simply use the default parameters for our softmax model. Customization of parameters like learning rate, batch size, and number of epochs is described in Tutorial 4 (in development) and is recommended for performance-critical applications.

In [8]:
repaired = session.repair()

Time for Domain Pruning: 3.69091701508

INSERT INTO Thread_9_0852044601848( SELECT              init_flat.vid as vid, init_flat.domain_id AS assigned_val,             '1' AS feature,             1 as count            FROM Observed_Possible_values_clean_0852044601848 AS init_flat WHERE (init_flat.attr_name = 'City' OR init_flat.attr_name = 'Stateavg' OR init_flat.attr_name = 'ZipCode' OR init_flat.attr_name = 'State' OR init_flat.attr_name = 'PhoneNumber' OR init_flat.attr_name = 'ProviderNumber' OR init_flat.attr_name = 'MeasureName' OR init_flat.attr_name = 'MeasureCode' OR init_flat.attr_name = 'Condition'));
INSERT INTO Thread_7_0852044601848(SELECT postab.vid as vid, postab.domain_id AS assigned_val, 2 AS feature,   count(t2.__ind) as count   FROM Init_0852044601848 as t1 ,Init_0852044601848 as t2,Possible_values_clean_0852044601848 as postab WHERE (postab.tid = t1.__ind AND postab.attr_name ='ZipCode' AND postab.attr_val=t2.ZipCode AND  t1.__ind < t2.__ind AND  t1.City<>t2.City) G

Time for Featurization: 1.17093801498



  6%|▌         | 3/50 [00:00<00:04,  9.81it/s]

Epoch 1, cost = 0.321568
Epoch 2, cost = 0.251864
Epoch 3, cost = 0.198702


 12%|█▏        | 6/50 [00:00<00:03, 11.02it/s]

...

Epoch 34, cost = 0.057729
Epoch 35, cost = 0.057728
Epoch 36, cost = 0.057727
Epoch 37, cost = 0.057727
Epoch 38, cost = 0.057726
Epoch 39, cost = 0.057726
Epoch 40, cost = 0.057726
Epoch 41, cost = 0.057726
Epoch 42, cost = 0.057725


100%|██████████| 50/50 [00:01<00:00, 29.95it/s]

Epoch 43, cost = 0.057725
Epoch 44, cost = 0.057725
Epoch 45, cost = 0.057725
Epoch 46, cost = 0.057725
Epoch 47, cost = 0.057725
Epoch 48, cost = 0.057725
Epoch 49, cost = 0.057725
Epoch 50, cost = 0.057725
Time for Training Model: 1.90788698196



INSERT INTO Thread_12_0852044601848( SELECT              init_flat.vid as vid, init_flat.domain_id AS assigned_val,             '1' AS feature,             1 as count            FROM Observed_Possible_values_dk_0852044601848 AS init_flat WHERE (init_flat.attr_name = 'City' OR init_flat.attr_name = 'Stateavg' OR init_flat.attr_name = 'ZipCode' OR init_flat.attr_name = 'State' OR init_flat.attr_name = 'PhoneNumber' OR init_flat.attr_name = 'ProviderNumber' OR init_flat.attr_name = 'MeasureName' OR init_flat.attr_name = 'MeasureCode' OR init_flat.attr_name = 'Condition'));
INSERT INTO Thread_13_0852044601848(SELECT postab.vid as vid, postab.domain_id AS assigned_val, 2 AS feature,   count(t2.__ind) as count   FROM Init_0852044601848 as t1 ,Init_0852044601848 as t2,Possible_values_dk_0852044601848 as postab WHERE (postab.tid = t1.__ind AND postab.attr_name ='ZipCode' AND postab.attr_val=t2.ZipCode AND  t1.__ind < t2.__ind AND  t1.City<>t2.City) GROUP BY postab.vid, postab.domain_id);
INSER

INSERT INTO Thread_13_0852044601848(SELECT postab.vid as vid, postab.domain_id AS assigned_val, 26 AS feature,   count(t2.__ind) as count   FROM Init_0852044601848 as t1 ,Init_0852044601848 as t2,Possible_values_dk_0852044601848 as postab WHERE (postab.tid = t1.__ind AND postab.attr_name ='Stateavg' AND postab.attr_val<>t2.Stateavg AND  t1.__ind < t2.__ind AND  t1.ProviderNumber=t2.ProviderNumber AND  t1.MeasureCode=t2.MeasureCode) GROUP BY postab.vid, postab.domain_id);
INSERT INTO Thread_12_0852044601848(SELECT postab.vid as vid, postab.domain_id AS assigned_val, 27 AS feature,   count(t1.__ind) as count   FROM Init_0852044601848 as t1 ,Init_0852044601848 as t2,Possible_values_dk_0852044601848 as postab WHERE (postab.tid = t2.__ind AND postab.attr_name = 'Stateavg' AND t1.Stateavg<>postab.attr_val AND  t1.__ind < t2.__ind AND  t1.ProviderNumber=t2.ProviderNumber AND  t1.MeasureCode=t2.MeasureCode) GROUP BY postab.vid, postab.domain_id);
INSERT INTO Thread_14_0852044601848(SELECT post

As we can see, our repaired dataset has effectively removed large numbers of the 'x' characters

In [9]:
repaired = repaired.withColumn("__ind", repaired["__ind"].cast("int"))
repaired.sort('__ind').select('City').show(15)

+----------+
|      City|
+----------+
|BIRMINGHAM|
|BIRMINGHAM|
|BIRMINGHAM|
|BIRMINGHAM|
|BIRMINGHAM|
|BIRMINGHAM|
|BIRMINGHAM|
|BIRMINGHAM|
| SHEFFIELD|
| SHEFFIELD|
| SHEFFIELD|
| SHEFFIELD|
| SHEFFIELD|
| SHEFFIELD|
| SHEFFIELD|
+----------+
only showing top 15 rows



### Performance Evaluation

Since this is a research dataset, a clean version is available for us to compare our results to. 

In [10]:
session.compare_to_truth("data/groundtruth.csv")

The precision that we have is :0.987947882736
The recall that we have is :0.777272727273
